**Автокорректор ошибок на Python**

In [1]:
# Импортируй и властвуй
%pylab inline
import re
import math
import string
from collections import Counter
import requests

/opt/sagemath-9.0/local/lib/python3.7/site-packages/matplotlib/font_manager.py:281: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


Populating the interactive namespace from numpy and matplotlib


/opt/sagemath-9.0/local/lib/python3.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['histogram', 'ceil', 'cos', 'piecewise', 'det', 'gcd', 'loads', 'rank', 'median', 'diff', 'power', 'copy', 'mean', 'plot', 'floor', 'NaN', 'e', 'sin', 'colors', 'uniform', 'SU', 'table', 'real', 'transpose', 'load', 'arctan', 'arccos', 'sample', 'product', 'add', 'log2', 'binomial', 'heaviside', 'Infinity', 'arctanh', 'tanh', 'arrow', 'norm', 'sign', 'hold', 'imag', 'arctan2', 'xkcd', 'log', 'arcsin', 'conjugate', 'randint', 'cosh', 'tan', 'var', 'shuffle', 'laplace', 'magma', 'gamma', 'solve', 'seed', 'colormaps', 'choice', 'pi', 'flatten', 'sqrt', 'arcsinh', 'multinomial', 'arccosh', 'hypergeometric', 'lcm', 'sinh', 'trace', 'mod', 'text', 'sum', 'save', 'show', 'matrix', 'std', 'exp', 'random', 'prod', 'euler_gamma', 'beta']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Для начала немного теории. 
Пусть дано слово, будем пытаться отыскать слово, в котором с наибольшей вероятностью исправлены допущенные ошибки (если ошибок нет, то таким словом будет данное). Разумеется, мы не сможем гарантировать 100% исправления всех ошибок. (Например, если нам дано слово «пак», то правильным будет слово «паз» или «парк» ?), именно поэтому мы используем вероятностный (или другими словами стохастический) подход. 
Будем говорить, что мы пытаемся выбрать такое слово c из всех возможных слов-исправлений, что вероятность появления именно слова c при данном слове w будет максимальна:

argmaxc P(c|w)


Согласно теореме Байеса - выражение, записанное выше, эквивалентно следующему выражению:

***argmaxc P(w|c) P(c) / P(w)***

Поскольку P(w) одинакова для всех c мы можем отбросить P(w), что даст нам:

***argmaxc P(w|c) P(c)***


В этом выражении присутствуют три части.  
Справа налево:

***P(c)*** – вероятность появления слова c (частотность употребления c). Эта вероятность обусловлена самим языком (точнее моделью языка). Иначе говоря, P(c) определяет как часто c встречается в текстах на русском языке. P(«превед») будет достаточно высока, тогда как P(«благоденствовать») будет меньше, а P(«ыгввыцшы») будет около нуля.

***P(w|c)*** – вероятность того, что автор опечатался и написал w, хотя имел в виду c. По сути дела эта вероятность обусловлена частотностью тех или иных ошибок в языке (и называется моделью ошибок языка).

***argmaxc*** – оператор, перебирающий все возможные c в поиске наиболее (вероятнее всего) подходящего из них (т.е. данный оператор ищет такое допустимое c, которе максимизирует условную вероятность появления w при данном c).

Может возникнуть очевидный вопрос – зачем мы преобразовали простое выражение «argmaxc P(c|w)» с помощью какого-то Байеса в более сложное выражение, в котором используются аж две языковые модели, вместо одной? Дело в том, что P(c|w) учитывает в себе сразу обе языковых модели, поэтому очевидно, что проще выделить эти модели и работать с ними по отдельности. Предположим у нас есть слово с опечаткой – «езать», это может быть как «ехать», так и «резать». Для какого из исправлений P(c|w) будет максимально ? Оба исправления имеют примерно одинаковую частотность в русском языке. Хорошо допустим «х» и «з» близко расположены в русской раскладке клавиатуры и это повышает вероятность варианта «ехать», но это не повод, чтоб отбрасывать «резать», ведь «е» и «р» тоже близки. Поэтому лучше не рассматривать P(c|w) как единую величину, ибо нам приходится учитывать и частность исправления c и вероятность исправления c для данной опечатки в w. Удобнее работать с этими двумя вероятностями по отдельности.

**Начнем с P(c)**. Мы читаем большой текстовый файл, russian.txt, в котором записано много слов.

После этого мы извлекаем отдельные слова из файла 


In [2]:
TEXT = open('russian.txt')
len(TEXT)

FileNotFoundError: [Errno 2] No such file or directory: 'russian.txt'

In [ ]:
def tokens(text):
    """Возвращает список токенов (подряд идущих буквенных последовательностей) в тексте. 
       Текст при этом приводится к нижнему регистру."""
    return re.findall(r'[а-я]+', text.lower()) 

In [ ]:
WORDS = tokens(TEXT)
len(WORDS)

Хочу заметить, что сейчас слова появляются в нашем списке в том порядке, как они располагались в файле

***Модель: Мешок слов (aka Bag of Words)***

Мы создали список *WORDS* - список слов в том порядке, как они следуют в *TEXT*. Мы можем использовать этот список в качестве порождающей модели (generative model) текста. Язык - очень сложная штука и мы создаем крайне упрощенную модель языка, которая может ухватить часть этой сложной структуры. 

В модели мешка слов , мы полностью игнорируем порядок слов, зато соблюдаем их частоту. Представить это можно себе так: вы берете все слова текста и забрасываете их в мешок. Теперь, если вы хотите сгенерировать предложение с помощью этого мешка, вы просто трясете его(слова там перемешиваются) и достаете указанное количество слов по одному (мешок непрозрачный, так что слоа вы достаете наугад). Почти наверное полученное предложение будет грамматически некорректным, но слова в этом предложении будут в +- правильной пропорции (более частые будут встречаться чаще, более редкие - реже). Вот функция, которая сэмплирует(от англ. sample) предложение из n слов с помощью нашего мешка:

In [ ]:
def sample(bag, n=10):
    "Sample a random n-word sentence from the model described by the bag of words."
    return ' '.join(random.choice(bag) for _ in range(n))

In [ ]:
sample(WORDS)

Другое представление мешка слов - Counter. Это словарь, состоящий из пар {'слово': кол-во вхождений слова в текст}. Например,

In [ ]:
Counter(tokens('Между нами провода, Города да да да. Я сказал иди сюда, И ты сказала: «Да, да, да..»'))

Counter очень похож на словарь из Python - тип dict , но у него есть ряд дополнительных методов. Давайте завернем в Counter наш список слов WORDS и посмотрим, что получится:


In [ ]:
COUNTS = Counter(WORDS)

print(COUNTS.most_common(10))

In [ ]:
for w in tokens('самые редкие слова: Крыжить, Михрютка, Драдедамовый '):
    print(COUNTS[w], w)

В 1935, лингвист Джордж Ципф отметил, что в любом большом тексте n-тое наиболее часто встречающееся слово появляется с частотой ~ 1/n от частоты наиболее часто встречающегося слова. Это наблюдение получило название Закона Ципфа, несмотря на то, что Феликс Ауэрбах заметил это еще в 1913 году. Если нарисовать частоты слов, начиная от самого часто встречающегося, на log-log-графике, они должны приблизительно следовать прямой линии, если закон Ципфа верен. 

In [ ]:
M = COUNTS['и']
yscale('log'); 
xscale('log'); 
title('Частота n-того наиболее частого слова и линия 1/n.')
plot([c for (w, c) in COUNTS.most_common()])
plot([M/i for i in range(1, len(COUNTS)+1)]);

***Задача: Проверка Правописания***

Для данного слова w нужно найти наиболее вероятную правку *c = correct(w).*

**Подход:** Найти все кандидаты c, достаточно близкие к w. Выбрать наиболее вероятный из них.

Осталось понять, что такое близкие и наиболее вероятный.

Применим наивный подход: всегда будем брать более близкое слово, если проверки на близость недостаточно, берем слово с максимальной частотой из WORDS. 
Сейчас мы будем измерять близость с помощью расстояния Левенштейна: минимального необходимого количества удалений, перестановок, вставок, и замен символов, необходимых чтобы одно слово превратить в другое. Конечно же это не единственный возможный подход. Методом проб и ошибок можно понять, что поиск слов в пределах расстояния 2 уже даст пристойные результаты (или можно почитать в литературе). Тогда остается определить функцию *correct(w):*

In [ ]:
def correct(word):
    "Поиск лучшего исправления ошибки для данного слова."
    # предрассчитать edit_distance==0, затем 1, затем 2; в противном случае оставить слово "как есть".
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=COUNTS.get)

Функции known и edits0 простые; функция edits2 iлегко получается из функции edits1:

In [ ]:
def known(words):
    "Вернуть подмножество слов, которое есть в нашем словаре."
    return {w for w in words if w in COUNTS}

def edits0(word): 
    "Вернуть все строки, которые находятся на edit_distance == 0 от word (т.е., просто само слово)."
    return {word}

def edits2(word):
    "Вернуть все строки, которые находятся на edit_distance == 2 от word."
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [ ]:
def edits1(word):
    "Возвращает список всех строк на расстоянии edit_distance == 1 от word."
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    "Возвращает список всех возможных разбиений слова на пару (a, b)."
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьяюя'

In [ ]:
splits('атец')

In [ ]:
len(alphabet)

In [ ]:
print(edits0('атец'))

In [ ]:
print(edits1('атец'))

In [ ]:
print(len(edits2('атец')))

In [ ]:
tokens('Ана ни зочит в шшколу.')

In [ ]:
list(map(correct, tokens('Ана ни зочит в шшколу.')))

***Теория: От счетчика слов к вероятностям последовательностей слов***

Нам нужно научиться подсчитывать вероятности слов, P(w). Делать мы это будем с помощью функции pdist, которая на вход принимает Counter (наш мешок слов) и возвращает функцию, выполняющую роль вероятностного распределения на множестве всех возможных слов. В вероятностном распределении вероятность каждого слова лежит между 0 и 1, и сложение вероятностей всех слов дает 1.

In [ ]:
def pdist(counter):
    "Превращает частоты из Counter в вероятностное распределение."
    N = sum(list(counter.values()))
    return lambda x: counter[x]/N

P = pdist(COUNTS)

In [ ]:
for w in tokens('То мать пирогов напечет, то бабушка с булочками приедет'):
    print(P(w), w)

Итак, что же такое вероятность последовательности слов? Используем определение совместной вероятности:
P(w_1 ... w_n) = P(w_1)*P(w_2|w_1)*P(w_3|w_1w_2)...*...P(w_n|w_1...w_n-1)

Модель мешка слов подразумевает, что каждое слово из мешка достается независимо от других. Это дает нам упрощенную аппроксимацию:
P(w_1 ... w_n) = P(w_1)*P(w_2)*P(w_3)...*...P(w_n)

Известный статистик Джордж Бокс сказал Все модели неверны, но некоторые полезны.

Как же нам посчитать P(w_1 ... w_n)? Мы будем использовать другое название, чтобы не обманывать себя, Pwords вместо P, и посчитаем ее как произведение индивидуальных вероятностей:

In [ ]:
def Pwords(words):
    "Вероятности слов, при условии, что они независимы."
    return product(P(w) for w in words)

def product(nums):
    "Перемножим числа.  (Это как `sum`, только с умножением.)"
    result = 1
    for x in nums:
        result *= x
    return result

In [ ]:
tests = ['Это тест', 
         'Это необычный текст',
         'Это эпико-концентрированный тест']

for test in tests:
    print(Pwords(tokens(test)), test)

ВОУ—кажется, присвоить последнюю вероятность 0, неправильно; Она просто должна быть маленькой. К этому вернемся попозже. Ну а другие вероятности кажутся +- адекватными.

***Задача: Разбиение слов на сегменты***
Ситуации, когда слова пишутся слитно (по ошибке или нет)


Подход 2: Делаем одно разбиение - на первое слово и все остальное. Если предположить, что слова независимы, можно максимизировать вероятность первого слова + лучшего разбиения оставшихся букв.

<code>
assert segment('choosespain') == ['choose', 'spain']
segment('choosespain') ==
   max(Pwords(['c'] + segment('hoosespain')),
       Pwords(['ch'] + segment('oosespain')),
       Pwords(['cho'] + segment('osespain')),
       Pwords(['choo'] + segment('sespain')),
       ...
       Pwords(['choosespain'] + segment('')))
</code>

Чтобы сделать это хоть сколько-нибудь эффективным, нужно избежать слишком большого числа пересчетов оставшейся части слова. Это можно сделать или с помощью динамического программирования или с помощью мемоизации aka кэширования. Кроме того, для первого слова не обязательно брать все возможные варианты разбиений - мы можем установить максимальную длину. Какой она должна быть? Чуть большей, чем длина самого длинного слова, которое мы видели.

In [ ]:
def memo(f):
    "Запомнить результаты исполнения функции f, чьи аргументы args должны быть хешируемыми."
    cache = {}
    def fmemo(*args):
        if args not in cache:
            cache[args] = f(*args)
        return cache[args]
    fmemo.cache = cache
    return fmemo

In [ ]:
max(len(w) for w in COUNTS)

In [ ]:
def splits(text, start=0, L=20):
    "Вернуть список всех пар (a, b); start <= len(a) <= L."
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]

In [ ]:
print(splits('word'))
print(splits('reallylongtext', 1, 4))

In [ ]:
@memo
def segment(text):
    "Вернуть список слов, который является наиболее вероятной сегментацией нашего текста."
    if not text: 
        return []
    else:
        candidates = ([first] + segment(rest) 
                      for (first, rest) in splits(text, 1))
        return max(candidates, key=Pwords)

In [ ]:
segment('фотоальбом')

In [ ]:
decl = ('Сложноепредложениеэтосинтаксическаяконструкциясостоящаяиздвухиболеепростыхпредложенийсвязанныхпосмыслуиинтонационноспомощьюсочинительнойподчинительнойилибессоюзнойсвязи.')

In [ ]:
print(segment(decl))

***Насколько дорого превращать одно слово в другое?***

Динамическое программирование позволяет разбить задачу на подзадачи, решив которые можно скомпоновать финальное решение. Мы будем пытаться превратить строку source[0..i] в строку target[0..j], мы сосчитаем все возможные комбинации подстрок substrings[i, j] и рассчитаем их edit_distance до нашей исходной. Мы будем сохранять результаты в таблицу и переиспользовать их для расчета дальнейших изменений.


In [ ]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Input: 
        source: строка-исходник
        target: строка, в которую мы должны исходник превратить
        ins_cost: цена вставки
        del_cost: цена удаления
        rep_cost: цена замены буквы
    Output:
        D: матрица размера len(source)+1 на len(target)+1 содержащая минимальные расстояния edit_distance
        med: минимальное расстояние edit_distance (med), необходимое, 
        чтобы превратить строку source в строку target
    '''
    # стоимость удаления и вставки = 1
    m = len(source)
    n = len(target)

    # Заткнем нашу матрицу нулями
    D = np.zeros((m+1, n+1), dtype=int) 
    
    # Заполним первую колонку
    for row in range(1,m+1): 
        D[row,0] = D[row-1,0] + del_cost
        
    # Заполним первую строку
    for col in range(1,n+1): 
        D[0,col] = D[0,col-1] + ins_cost
        
    # Теперь пойдем от 1 к m-той строке
    for row in range(1,m+1): 
        
        # итерируемся по колонкам от 1 до n
        for col in range(1,n+1):
            
            # r_cost - стоимость замены
            r_cost = rep_cost
            
            # Совпадает ли буква исходного слова из предыдущей строки
            # с буквой целевого слова из предыдущей колонки, 
            if source[row-1] == target[col-1]:
                # Если они не нужны, то замена не нужна -> стоимость = 0
                r_cost = 0
                
            # Обновляем значение ячейки на базе предыдущих значений 
            # Считаем D[i,j] как минимум из трех возможных стоимостей (как в формуле выше)
            D[row,col] = min([D[row-1,col]+del_cost, D[row,col-1]+ins_cost, D[row-1,col-1]+r_cost])
          
    # установить edit_distance в значение из правого нижнего угла
    med = D[m,n]
    

    return D, med

In [ ]:
import pandas as pd

source =  'кроты'
target = 'киты'
matrix, min_edits = min_edit_distance(source, target)

print("Расстояние: ",min_edits, "\n")

idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

Нам мало миллионов слов в "обучающей выборке" давайте перейдем к МИЛЛИАРДАМ слов. Получив такой огромный объем информации, можно перейти к анализу пар последоваительных слов, не ожидая, что вероятности слишком часто будут обнуляться (представьте себе, сколько в языке может быть грамматически корректных сочетаний из двух слов). Мы вновь позаимствуем уже собранные данные у мистера Норвига. Лежат они на его сайте в формате "word \t count" для отдельных слов и в формате "word1 word2 \t count" для биграмм. Считаем их и упакуем в наши словари с вероятностями:

***Валидация***

До настоящего момента мы пытались интуитивно оценить результаты нашей работы. Тем не менее, никаких численных оценок качества мы пока не получили. Важно понимать, что без четких метрик слова "плохо"/"хорошо" не имеют никакого смысла. Более того - мы даже не можем четко ответить, было ли наше обновление модели в лучшую сторону или худшую. Обычно при построении неких прогностических моделей данные разбиваются на три части:

**Обучающая выборка**: То, что мы использовали для создания модели исправления ошибок; У нас это был файл russian.txt file.
**Тестовая выборка**: Набор данных, который можно использовать для оценки качества вашей модели по ходу разработки.
**Валидационная выборка**: Набор данных, который мы используем для оценки работы программы после того как программа готова. Тестовая выборка для этого быть использована не может—Стоит разработчику посмотреть на результаты на тестовой выборке, она уже "испорчена". В принципе, программист может изменить программу так, чтобы она "подгонялась" под тестовую выборку, а это будет "переобучением". Вот почему нам нужен отдельный набор тестов, который рассматривается только после завершения разработки..

Для нашей программы обучающая выборка - словарь слов. Сделаем валидационную выборку.

In [ ]:
def test_segmenter(segmenter, tests):
    "Оценка сегментатора на тестовых данных; вывести на печать ошибки; вернуть долю верно разбитого."
    return sum([test_one_segment(segmenter, test) 
               for test in tests]), len(tests)

def test_one_segment(segmenter, test):
    words = tokens(test)
    result = segmenter(''.join(words))
    correct = (result == words)
    if not correct:
        print('expected', words)
        print('got     ', result) 
    return correct

proverbs = ("""тут какой-то тест"""
  .splitlines())

In [ ]:
test_segmenter(segment, proverbs)

**Место для анекдота про Лапласа**

Однажды французского математика Лапласа спросили: "Какова вероятность того, что Солнце завтра взойдет?". Из данных, что оно из  ближайших дней взошло n раз следует оценка максимального правдоподобия n/n = 1. Но Лапласу хотелось чуть сбалансировать оценку на шанс того, что завтра Солнце может и не взойти, поэтому он дал оценку (n+1)/(n+2).

То, что мы знаем, ограничено, а то, чего мы не знаем,-бесконечно
— Пьер Симон Лаплас, 1749-1827

In [ ]:
def pdist_additive_smoothed(counter, c=1):
    """Вероятность слова, при условии данных из Counter'a.
    добавляем c к частоте каждого слова + слово 'unknown'."""
    N = sum(list(counter.values()))          # суммарное кол-во слов
    Nplus = N + c * (len(counter) + 1) # кол-во слов + сглаживание
    return lambda word: (counter[word] + c) / Nplus 

P1w = pdist_additive_smoothed(COUNTS1)

Теперь еще одна проблема ... у нас появились незнакомые слова с ненулевой вероятностью. А что если 10-12 - приемлемая вероятность для слов нашего текста: то есть, если я читаю новый текст, вероятность того, что следующее слово мне незнакомо, может быть порядка 10-12. Но если я случайно генерирую 20-буквенный последовательности, вероятность того, что одна из них будет реальным словом намного меньше чем 10-12.

У нас две проблемы:

Во-первых, у нас нет четкой модели для неизвестных слов. Мы говорим "неизвестное слово", но не различаем более вероятные неизвестные слова и менее вероятные неизвестные слова. Ну, например, вероятнее ли 8-буквенное неизвестное слово чем 20-буквенное неизвестное слово?

Во-вторых, мы не берем в расчет информацию из частей неизвестных слов. Например, "unglobulate" явно должно быть более вероятным чем "zxfkogultae".

Для нашего следующего подхода мы используем идеи метода Гуда - Тьюринга. Он оценивает вероятности слов, не встретившихся в нашем Counter'е, на основании вероятностей слов, встретившихся единожды (Можно туда же подключить вероятности для встретившихся 2 раза и т.д.).

Итак, сколько слов встретилось 1 раз в COUNTS? (В COUNTS1 ни одного такого слова нет.) И какие длины у этих слов? Давайте посмотрим:


In [ ]:
singletons = (w for w in COUNTS if COUNTS[w] == 1)
lengths = list(map(len, singletons))
Counter(lengths).most_common()

In [ ]:
hist(lengths, bins=len(set(lengths)));

Длины таких слов распределены похоже на нормальное распределение :)

In [ ]:
def pdist_good_turing_hack(counter, onecounter, base=1/26., prior=1e-8):
    """Вероятность слова при условии данных из счетчика.
    Для неизвестных слов, смотрим на слова, встретившиеся единожды из onecounter, 
    вероятность выбираем, основываясь на длине.
    Воспользуемся идеей метода Гуда-Тьюринга(полностью мы его здесь не реализуем).
    prior -добавочный фактор, который сделает неизвестные слова менее вероятными.
    base -то, насколько мы уменьшаем вероятность за длину слова больше максимального."""
    N = sum(list(counter.values()))
    N2 = sum(list(onecounter.values()))
    lengths = list(map(len, [w for w in onecounter if onecounter[w] == 1]))
    ones = Counter(lengths)
    longest = max(ones)
    return (lambda word: 
            counter[word] / N if (word in counter) 
            else prior * (ones[len(word)] / N2 or 
                          ones[longest] / N2 * base ** (len(word)-longest)))
#Переопределим P1w
P1w = pdist_good_turing_hack(COUNTS1, COUNTS)

In [ ]:
segment.cache.clear()
segment('какой-то сегмент')

***Задача: Что если слово находится очень далеко по edit_distance, но звучит точно так же?***
Часто можно встретить ошибки в текстах, вызванные неграмотным написанием слов. Особенно часто это происходит в случае иностранных фамилий или транслитерированной терминологии. Обычно в таких случаях в пример приводят написание фамилии

Для такого случая можно использовать следующую методологию. Давайте привлечем лингвистов и составим правила, которые одинаково звучащим словам будут ставить в соответствие один и тот же код. Допустим, с помощью лингвистов мы такой алгоритм придумали. Тогда дальнейшие наши действия таковы:

1) Сделать словарь с вероятностями слов (как мы делали из мешка слов)

2) Сделать словарь соответствий код слова -> слово (с помощью того самого алгоритма от лингвистов). 
    Если есть в списке есть слова с одинаковым кодом, выбирать будем наиболее частое слово.

3) Сделаем аналогичный edit_distance алгоритм на множестве кодов слов

4) Найдя соответствующую замену для слова в виде его кода, восстановим слово с помощью словаря из пункта 2

Алгоритм, про который мы поговорим, называется Double Metaphone.

In [ ]:
from metaphone import doublemetaphone

Алгоритм возвращает кортеж из двух возможных фонетических кодов слова. Правило такое:

 (Primary Key = Primary Key) = Идеальное совпадение
 
 (Secondary Key = Primary Key) = Совпадение
 
 (Primary Key = Secondary Key) = Совпадение
 
 (Alternate Key = Alternate Key) = Совпадение +-
 


# THE END

У Норвига в статье код занимает 21 строчку: 


In [ ]:
import re, collections
def words(text): return re.findall('[a-z]+', text.lower())
def train(features):
    model = collections.defaultdict(lambda: 1)
        for f in features:
            model[f] += 1
        return model
NWORDS = train(words(file('big.txt').read()))
alphabet = 'abcdefghijklmnopqrstuvwxyz'
def edits1(word):
    n = len(word)
    return set( [word[0:i]+word[i+1:] for i in range(n)] +                      # deletion
                [word[0:i]+word[i+1]+word[i]+word[i+2:] for i in range(n-1)] +   # transposition
                [word[0:i]+c+word[i+1:] for i in range(n) for c in alphabet] +    # alteration
                [word[0:i]+c+word[i:] for i in range(n+1) for c in alphabet])    # insertion
def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)
def known(words): return set(w for w in words if w in NWORDS)
def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=lambda w: NWORDS[w])


возможно так переделано для русского:


In [ ]:
import re, collections
def words(text): return re.findall('[а-я]+', text.lower())
def train(features):
    model = collections.defaultdict(lambda: 1)
        for f in features:
            model[f] += 1
        return model
    
NWORDS = train(words(file('russian.txt').read()))
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
def edits1(word):
    n = len(word)
    return set( [word[0:i]+word[i+1:] for i in range(n)] +                      # deletion
                [word[0:i]+word[i+1]+word[i]+word[i+2:] for i in range(n-1)] +   # transposition
                [word[0:i]+c+word[i+1:] for i in range(n) for c in alphabet] +    # alteration
                [word[0:i]+c+word[i:] for i in range(n+1) for c in alphabet])    # insertion
def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)
def known(words): return set(w for w in words if w in NWORDS)
def correct(word):
    candidates = known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates, key=lambda w: NWORDS[w])


Использовать этот код нужно следующим образом –



In [ ]:
>>> correct('speling')
'spelling'
>>> correct('korrecter')
'corrector'